<a href="https://colab.research.google.com/github/fsilvao/ia_public/blob/main/Tarea2_1_Fredy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from itertools import chain

# Determinar el device que se va a utilizar (GPU o CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device} device')

# Crear los datasets de entrenamiento y prueba, usando torchvision
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_set = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Crear los loaders de los datasets de entrenamiento y prueba
train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
test_loader = DataLoader(test_set, batch_size=100, shuffle=False)

def etiqueta_ropa(etiqueta):
    output_mapping = {
        0: "T-shirt/Top", 1: "Trouser", 2: "Pullover",
        3: "Dress", 4: "Coat", 5: "Sandal",
        6: "Shirt", 7: "Sneaker", 8: "Bag", 9: "Ankle Boot"
    }
    input = (etiqueta.item() if type(etiqueta) == torch.Tensor else etiqueta)
    return output_mapping[input]

# Definición de la clase FashionCNN
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64 * 7 * 7, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(-1, 64 * 7 * 7)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

# Instanciamos el modelo y lo pasamos al device
model = FashionCNN().to(device)

# Función de pérdida y optimizador
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
error = nn.CrossEntropyLoss()
num_epochs = 5

# Entrenamiento del modelo
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(train)
        loss = error(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluación del modelo en el conjunto de prueba
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        test = Variable(images.view(100, 1, 28, 28))
        outputs = model(test)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on test images: {100 * correct / total:.2f}%')

Using cpu device


100%|██████████| 26421880/26421880 [00:01<00:00, 14930574.37it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 273419.73it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5015243.71it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 3326494.68it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Epoch [1/5], Loss: 0.6429
Epoch [2/5], Loss: 0.4720
Epoch [3/5], Loss: 0.5863
Epoch [4/5], Loss: 0.4226
Epoch [5/5], Loss: 0.3584
Accuracy of the model on test images: 85.54%
